__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/pandasklar/blob/master/jupyter/52_Rank_Rows.ipynb)__

## Rank Rows
* `rank`: Select the max row per group. Or the min.<br>
   Or mark the rows instead of selecting them. 

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 21:51:55


In [2]:
import numpy      as np
import pandas     as pd 
import bpyth      as bpy

# pandasklar
try:
    import pandasklar as pak 
except ImportError as e:
    !pip install pandasklar
    import pandasklar as pak   
    
# verbose
pak.Config.set('VERBOSE', True)

# copy_on_write
pd.set_option("mode.copy_on_write", True)

import time
from pandas._testing import assert_frame_equal

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions



In [3]:
# Create test data
# cols_group = ['AA']     # Very few very large groups
# cols_group = ['A']      # Few large groups
# cols_group = ['B']      # Many small groups

np.random.seed(10)
anz = 100000 #0
df = pak.zufallsdaten(anz)
df['AA'] = df.A.apply(lambda x: int(x/10)*10 )
df['AA'] = pak.change_datatype(df['AA'] )
df = pak.move_cols(df,'AA')
df = pak.rename_col(df,'C','other_data')
df = pak.rename_col(df,'D','score')
df
#pak.analyse_cols(df)

AA                   --> pd.Int8   


,AA,A,B,other_data,score,E
0,40,40,97438,0.369,9.97,0.000000e+00
1,30,33,87940,0.186,5.87,1.610982e-09
2,30,32,95268,0.063,3.27,8.112630e-09
3,20,27,61350,0.551,5.84,1.294510e-08
4,30,33,32778,0.393,1.92,2.594935e-08
...,...,...,...,...,...,...
99995,30,33,52307,0.320,8.20,9.999999e-01
99996,30,32,17225,0.366,6.87,1.000000e+00
99997,30,31,72053,0.583,4.13,1.000000e+00
99998,20,29,97320,0.443,7.72,1.000000e+00


## rank()

In [4]:
?pak.rank

Signature:
pak.rank(
    df,
    col_score,
    cols_group=None,
    find='max',
    col_target='',
    on_conflict='first',
    verbose=None,
)
Docstring:
Select the max row per group. Or the min.
Or mark the rows instead of selecting them. 
* cols_group:   Name of the columns to be grouped by. None if no grouping needed.
* col_score:    Name of the column whose minimum or maximum is to be found.
* find:         'min' or 'max', default is 'max'.
* col_target:   Should a ranking column be added? If yes, then give the name here.
                If col_target is given, then the dataframe is returned completely, plus the new column.
                If no col_target is given, only rank 1 is returned.
* on_conflict:  How to rank the group of records that have the same value. 
                Possible values: 'min','max','average','dense' and 'first', see pandas rank.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/rank.py
Type:      function


In [5]:
# select ONE row with max score
ranked = pak.rank( df, col_score='score')
ranked

rank: 99999 rows less, now 1 rows


,AA,A,B,other_data,score,E
567,20,26,54706,0.479,9.99,0.000392


In [6]:
# select the rows with max score, grouped by AA
ranked = pak.rank( df, cols_group=['AA'], col_score='score')
ranked

rank: 99997 rows less, now 3 rows


,AA,A,B,other_data,score,E
567,20,26,54706,0.479,9.99,0.000392
2072,30,36,48311,0.574,9.99,0.005199
2298,40,40,55534,0.408,9.99,0.006389


In [7]:
# select the rows with min score, grouped by AA
ranked = pak.rank( df, cols_group=['AA'], col_score='score', find='min')
ranked

rank: 99997 rows less, now 3 rows


,AA,A,B,other_data,score,E
886,30,32,12017,0.863,0.0,0.000954
1058,20,25,63125,0.128,0.0,0.001361
50593,40,40,78253,0.662,0.0,0.484162


### mark instead of select

In [8]:
# Instead of selecting the top rows, we can mark all rows by it's rank.
# If no group is given, the one with max score in the whode DataFrame gets rank 1.
# This time we group by A.

df = pak.rank( df, col_score='score', col_target='rank')
df.sort_values(['rank'])

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
567,20,26,54706,0.479,9.99,0.000392,1
2072,30,36,48311,0.574,9.99,0.005199,2
2298,40,40,55534,0.408,9.99,0.006389,3
3865,20,20,21622,0.781,9.99,0.017924,4
5834,20,28,24588,0.672,9.99,0.040131,5
...,...,...,...,...,...,...,...
95513,30,30,74955,0.691,0.00,0.966787,99996
96222,30,32,85761,0.432,0.00,0.976343,99997
98138,30,32,30890,0.036,0.00,0.994204,99998
99439,30,37,83082,0.964,0.00,0.999472,99999


In [9]:
# Instead of selecting the top rows, we can mark all rows by it's rank.
# The one with max score in the group gets rank 1.
# This time we group by A.

df = pak.rank( df, cols_group=['A'], col_score='score', col_target='rank')
df.sort_values(['A','score'], ascending=[True,False])

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
3865,20,20,21622,0.781,9.99,0.017924,1
566,20,20,94494,0.075,9.98,0.000389,2
77384,20,20,52557,0.081,9.98,0.509063,3
44876,20,20,76498,0.624,9.97,0.483304,4
35531,20,20,82368,0.454,9.96,0.464379,5
...,...,...,...,...,...,...,...
44926,40,42,31006,0.253,0.11,0.483305,681
56713,40,42,25721,0.053,0.11,0.487606,682
46278,40,42,26719,0.590,0.06,0.483364,683
68928,40,42,14264,0.344,0.04,0.496892,684


### Deal with ambiguous results 

In [10]:
# In this dataset there are many records with max score.
mask = (df.score >= 9.99)  &  (df.AA==20)
df[mask].head(5)

,AA,A,B,other_data,score,E,rank
567,20,26,54706,0.479,9.99,0.000392,1
3865,20,20,21622,0.781,9.99,0.017924,1
5834,20,28,24588,0.672,9.99,0.040131,1
8084,20,24,77773,0.778,9.99,0.074937,1
11781,20,29,10303,0.086,9.99,0.149074,1


In [11]:
# on_conflict='first' is default.
df = pak.rank( df, cols_group=['AA'], col_score='score', col_target='rank')
df.sort_values(['AA','score'], ascending=[True,False])

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
567,20,26,54706,0.479,9.99,0.000392,1
3865,20,20,21622,0.781,9.99,0.017924,2
5834,20,28,24588,0.672,9.99,0.040131,3
8084,20,24,77773,0.778,9.99,0.074937,4
11781,20,29,10303,0.086,9.99,0.149074,5
...,...,...,...,...,...,...,...
92661,40,41,87336,0.393,0.03,0.913647,4075
97927,40,41,76769,0.471,0.03,0.992819,4076
74961,40,42,34412,0.743,0.02,0.500572,4077
44135,40,40,92031,0.237,0.01,0.483289,4078


In [12]:
# If you sort the dataframe before, you will get secondary ranking criterias.
# In this example the ranking criterias are score and B.
df = pak.rank( df.sort_values('B'), cols_group=['AA'], col_score='score', col_target='rank')
df.sort_values(['AA','score'], ascending=[True,False])

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
11781,20,29,10303,0.086,9.99,0.149074,1
70760,20,27,11885,0.398,9.99,0.497814,2
89646,20,22,15461,0.293,9.99,0.835732,3
26245,20,23,17271,0.158,9.99,0.409954,4
95488,20,27,20148,0.373,9.99,0.966432,5
...,...,...,...,...,...,...,...
97927,40,41,76769,0.471,0.03,0.992819,4075
92661,40,41,87336,0.393,0.03,0.913647,4076
74961,40,42,34412,0.743,0.02,0.500572,4077
44135,40,40,92031,0.237,0.01,0.483289,4078


In [13]:
# group by A, on_conflict='average' 
df = pak.rank( df, cols_group=['A'], col_score='score', col_target='rank', on_conflict='average')
a = df.sort_values(['A','score'], ascending=[True,False])
a

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
3865,20,20,21622,0.781,9.99,0.017924,1.0
77384,20,20,52557,0.081,9.98,0.509063,2.5
566,20,20,94494,0.075,9.98,0.000389,2.5
44876,20,20,76498,0.624,9.97,0.483304,4.0
86925,20,20,18985,0.791,9.96,0.752037,6.0
...,...,...,...,...,...,...,...
56713,40,42,25721,0.053,0.11,0.487606,681.5
44926,40,42,31006,0.253,0.11,0.483305,681.5
46278,40,42,26719,0.590,0.06,0.483364,683.0
68928,40,42,14264,0.344,0.04,0.496892,684.0


In [14]:
# group by A, on_conflict='min' 
df = pak.rank( df, cols_group=['A'], col_score='score', col_target='rank', on_conflict='min')
a = df.sort_values(['A','score'], ascending=[True,False])
a

rank: 0 rows less, now 100000 rows


,AA,A,B,other_data,score,E,rank
3865,20,20,21622,0.781,9.99,0.017924,1
77384,20,20,52557,0.081,9.98,0.509063,2
566,20,20,94494,0.075,9.98,0.000389,2
44876,20,20,76498,0.624,9.97,0.483304,4
86925,20,20,18985,0.791,9.96,0.752037,5
...,...,...,...,...,...,...,...
56713,40,42,25721,0.053,0.11,0.487606,681
44926,40,42,31006,0.253,0.11,0.483305,681
46278,40,42,26719,0.590,0.06,0.483364,683
68928,40,42,14264,0.344,0.04,0.496892,684
